In [75]:
import pandas as pd
import math
import numpy as np
import torch
from sklearn.model_selection import train_test_split

In [76]:
df = pd.read_excel("./data.xlsx")
df.columns

# first_transactions = {"sns_first_10_sum": [], "sns_first_0": []}
# for bought in df_open["sns_bought"]:
#     bought_lst = [float(x) for x in bought.split(" ")]
#     end = len(bought) if len(bought) < 10 else 10
#     first_0 = 0
    
#     for value in bought_lst[:end]:
#         if value == 0 or value == 0.0:
#             first_0 += 1
#         # else:
#         #     break
    
#     sum_10 = sum(bought_lst[:end])
#     first_transactions["sns_first_10_sum"].append(sum_10)
#     first_transactions["sns_first_0"].append(first_0)
            
# df_extra = pd.DataFrame(first_transactions)
# df = pd.merge(df_open, df_extra, left_index=True, right_index=True)  

Index(['id', 'pair', 'token_name', 'token_address', 'price_b', 'price_s',
       'token_age_b', 'token_age_s', 'transfers_b', 'transactions_b',
       'buys_m5_b', 'sells_m5_b', 'buys_h1_b', 'sells_h1_b', 'buys_h6_b',
       'sells_h6_b', 'buys_h24_b', 'sells_h24_b', 'buys_m5_s', 'sells_m5_s',
       'buys_h1_s', 'sells_h1_s', 'buys_h6_s', 'sells_h6_s', 'buys_h24_s',
       'sells_h24_s', 'volume_m5_b', 'volume_h1_b', 'volume_h6_b',
       'volume_h24_b', 'volume_m5_s', 'volume_h1_s', 'volume_h6_s',
       'volume_h24_s', 'price_change_m5_b', 'price_change_h1_b',
       'price_change_h6_b', 'price_change_h24_b', 'price_change_m5_s',
       'price_change_h1_s', 'price_change_h6_s', 'price_change_h24_s',
       'liquidity_b', 'liquidity_s', 'fdv_b', 'fdv_s', 'market_cap_b',
       'market_cap_s', 'is_telegram', 'is_twitter', 'is_website',
       'opening_date', 'closing_date', 'sns_bought', 'sns_sold',
       'sns_sum_bought', 'sns_sum_sold', 'sns_held_all', 'sns_sold_some',
       'sns_

In [77]:
drop_columns = [
    "id", "pair", "token_name", "token_address", 
    "price_b", "price_s", 'token_age_b', "token_age_s", 
    'buys_m5_b', 'sells_m5_b', 'buys_h1_b', 'sells_h1_b',
    "buys_h6_b", "sells_h6_b", "buys_h24_b", "sells_h24_b", 'buys_m5_s', 'sells_m5_s', 'buys_h1_s', 'sells_h1_s', 'buys_h6_s', 'sells_h6_s', 'buys_h24_s', 'sells_h24_s',
    'volume_h1_b', 'volume_h6_b', 'volume_h24_b', 'volume_m5_s', 'volume_h1_s', 'volume_h6_s', 'volume_h24_s',
    'price_change_h1_b', 'price_change_h6_b', 'price_change_h24_b', 'price_change_m5_s', 'price_change_h1_s', 'price_change_h6_s', 'price_change_h24_s',
    'liquidity_s', 'fdv_s', 'market_cap_b', 'market_cap_s',
    'tt_bought', 'tt_sold', 'sns_sold', 'sns_bought', 
    'opening_date', 'closing_date',
    'status', 'mode']
df = df.drop(drop_columns, axis = 1)
df.head()

,transfers_b,transactions_b,volume_m5_b,price_change_m5_b,liquidity_b,fdv_b,is_telegram,is_twitter,is_website,sns_sum_bought,...,tt_bought_01_less,tt_bought_5000_more,tt_sold_01_less,tt_sold_5000_more,tt_pnl_5000_more,tt_no_bought,tt_no_sold,tt_pnl_profit,tt_pnl_loss,PNL
0,624,235,"40891,56","-56,17","16581,54","27037,0",0,0,0,"16236,09",...,4,0,1,0,0,54,32,59,9,NaN
1,15911,3,"1518,39","10,15","15667,62","606356,0",1,1,1,"1836,0",...,0,0,0,0,0,0,2,0,0,NaN
2,77,243,"56203,32","-80,78","10790,43","11482,0",0,0,0,"21746,46",...,6,0,3,0,0,39,36,54,9,"-21,77"
3,7711,724,"9644,94","1261,0","0,11","36656,0",1,1,0,"369,73",...,10,0,4,1,1,10,9,11,72,"354,57"
4,9445,42,"1537,83","2,24","3132,01","663778,0",0,0,0,"1058,68",...,0,0,0,0,0,2,4,5,4,"-28,79"


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   transfers_b           431 non-null    int64 
 1   transactions_b        431 non-null    int64 
 2   volume_m5_b           431 non-null    object
 3   price_change_m5_b     431 non-null    object
 4   liquidity_b           431 non-null    object
 5   fdv_b                 431 non-null    object
 6   is_telegram           431 non-null    int64 
 7   is_twitter            431 non-null    int64 
 8   is_website            431 non-null    int64 
 9   sns_sum_bought        431 non-null    object
 10  sns_sum_sold          431 non-null    object
 11  sns_held_all          431 non-null    int64 
 12  sns_sold_some         431 non-null    int64 
 13  sns_sold_all          431 non-null    int64 
 14  sns_bought_01_less    431 non-null    int64 
 15  sns_bought_5000_more  431 non-null    in

In [79]:
df = df.dropna(subset=['transfers_b', 'PNL'])

In [80]:
#df["price_b"] = df["price_b"].replace(",",".", regex=True).astype(float)
#df["token_age_b"] = df["token_age_b"].replace(",",".", regex=True).astype(float)
df["liquidity_b"] = df["liquidity_b"].replace(",",".", regex=True).astype(float)
df["fdv_b"] = df["fdv_b"].replace(",",".", regex=True).astype(float)
df["volume_m5_b"] = df["volume_m5_b"].replace(",",".", regex=True).astype(float)
#df["volume_h1_b"] = df["volume_h1_b"].replace(",",".", regex=True).astype(float)
df["price_change_m5_b"] = df["price_change_m5_b"].replace(",",".", regex=True).astype(float)
#df["price_change_h1_b"] = df["price_change_h1_b"].replace(",",".", regex=True).astype(float)
df["sns_sum_bought"] = df["sns_sum_bought"].replace(",",".", regex=True).astype(float)
df["sns_sum_sold"] = df["sns_sum_sold"].replace(",",".", regex=True).astype(float)
df["tt_sum_bought"] = df["tt_sum_bought"].replace(",",".", regex=True).astype(float)
df["tt_sum_sold"] = df["tt_sum_sold"].replace(",",".", regex=True).astype(float)
df["PNL"] = df["PNL"].replace(",",".", regex=True).astype(float)

In [81]:
df = df[(df["transfers_b"] > 10) & (df["transfers_b"] < 5000)]

In [82]:
df.loc[df["PNL"] >= 60, "profit"] = 1
df.loc[df["PNL"] < 60, "profit"] = 0
df = df.drop(["PNL"], axis = 1)
df["profit"] = df["profit"].astype(int)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, 2 to 428
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   transfers_b           324 non-null    int64  
 1   transactions_b        324 non-null    int64  
 2   volume_m5_b           324 non-null    float64
 3   price_change_m5_b     324 non-null    float64
 4   liquidity_b           324 non-null    float64
 5   fdv_b                 324 non-null    float64
 6   is_telegram           324 non-null    int64  
 7   is_twitter            324 non-null    int64  
 8   is_website            324 non-null    int64  
 9   sns_sum_bought        324 non-null    float64
 10  sns_sum_sold          324 non-null    float64
 11  sns_held_all          324 non-null    int64  
 12  sns_sold_some         324 non-null    int64  
 13  sns_sold_all          324 non-null    int64  
 14  sns_bought_01_less    324 non-null    int64  
 15  sns_bought_5000_more  324 no

In [84]:
df.head()

,transfers_b,transactions_b,volume_m5_b,price_change_m5_b,liquidity_b,fdv_b,is_telegram,is_twitter,is_website,sns_sum_bought,...,tt_bought_01_less,tt_bought_5000_more,tt_sold_01_less,tt_sold_5000_more,tt_pnl_5000_more,tt_no_bought,tt_no_sold,tt_pnl_profit,tt_pnl_loss,profit
2,77,243,56203.32,-80.78,10790.43,1.148200e+04,0,0,0,21746.46,...,6,0,3,0,0,39,36,54,9,0
5,1437,1132,27623.38,-65.36,10178.58,9.844000e+03,0,0,0,13564.02,...,0,0,0,0,0,78,0,100,0,0
6,549,541,70778.82,-11.97,31485.19,9.531600e+04,0,0,0,35889.20,...,7,0,0,3,3,19,31,46,19,0
7,3233,1349,40298.75,-45.21,21024.38,4.243900e+04,0,0,0,7691.08,...,0,0,0,0,0,88,0,100,0,1
8,232,198,9192.38,3.97,364952.88,1.774991e+09,0,0,0,186533.15,...,32,0,0,0,0,3,80,11,8,0


In [85]:
df.describe()

,transfers_b,transactions_b,volume_m5_b,price_change_m5_b,liquidity_b,fdv_b,is_telegram,is_twitter,is_website,sns_sum_bought,...,tt_bought_01_less,tt_bought_5000_more,tt_sold_01_less,tt_sold_5000_more,tt_pnl_5000_more,tt_no_bought,tt_no_sold,tt_pnl_profit,tt_pnl_loss,profit
count,324.000000,324.000000,3.240000e+02,3.240000e+02,324.000000,3.240000e+02,324.000000,324.000000,324.000000,324.000000,...,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000
mean,1519.024691,868.938272,5.822021e+04,6.719556e+04,53275.477284,1.622425e+08,0.222222,0.240741,0.182099,39837.433580,...,6.691358,0.311728,0.277778,1.929012,1.518519,38.577160,24.543210,65.043210,4.450617,0.274691
std,1276.813923,924.424274,1.164510e+05,5.005990e+05,92869.882284,6.254589e+08,0.416383,0.428195,0.386523,59478.751517,...,10.401407,0.745503,0.731850,4.977773,4.563582,29.167026,28.765971,36.241766,7.243703,0.447049
min,12.000000,2.000000,0.000000e+00,-1.000000e+02,148.360000,3.340000e+02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,431.750000,228.250000,6.474773e+03,-1.755250e+01,11900.742500,2.063950e+04,0.000000,0.000000,0.000000,9500.475000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,28.750000,0.000000,0.000000
50%,1182.500000,543.500000,2.350955e+04,1.965000e+00,23895.155000,6.592100e+04,0.000000,0.000000,0.000000,18523.355000,...,1.000000,0.000000,0.000000,0.000000,0.000000,38.500000,11.000000,72.500000,0.000000,0.000000
75%,2317.250000,1120.500000,6.598601e+04,4.628250e+01,45234.820000,4.689454e+06,0.000000,0.000000,0.000000,40918.917500,...,13.000000,0.000000,0.000000,1.000000,0.000000,65.000000,49.000000,100.000000,7.000000,1.000000
max,4933.000000,4574.000000,1.299786e+06,4.543835e+06,480323.040000,3.825042e+09,1.000000,1.000000,1.000000,515220.650000,...,68.000000,6.000000,5.000000,31.000000,30.000000,100.000000,92.000000,100.000000,45.000000,1.000000


In [86]:
x = df.drop(columns=["profit"])
y = df["profit"].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [87]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

In [88]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [89]:
n_features = x_train.shape[1]
model = LogisticRegression(n_features)

optim = torch.optim.SGD(model.parameters(), lr=1)
criterion = torch.nn.BCELoss()

In [90]:
EPOCHS = 3

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f'Среднее значение функции потерь за эпоху {e}: {loss.data}')
    return model

In [91]:
model = train(model, optim, criterion, x_train, y_train)

Среднее значение функции потерь за эпоху 1: 53.409873962402344
Среднее значение функции потерь за эпоху 2: 71.42857360839844
Среднее значение функции потерь за эпоху 3: 71.42857360839844


In [92]:
def accuracy(model, x, y):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

plain_accuracy = accuracy(model, x_test, y_test)
print(f'Точность модели: {plain_accuracy}')

Точность модели: 0.23076923191547394


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

x = df.drop(columns=["profit"])
y = df["profit"].copy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создание и обучение модели логистической регрессии
model = LogisticRegression(solver='lbfgs', max_iter=100)
model.fit(X_train, y_train)

# Предсказание классов на тестовой выборке
y_pred = model.predict(X_test)

# Вычисление точности классификации
accuracy = accuracy_score(y_test, y_pred)
print("Точность классификации: {:.2f}%".format(accuracy * 100))

Точность классификации: 76.92%


/home/irina/toss a coin/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
